# [George McNinch](http://gmcninch.math.tufts.edu) Math 87 - Spring 2026

# Some more remarks on duality linear programs 

# Week 6



In our earlier discussions, we only talked about duality for linear programs in *standard form* -- i.e. only with
inequality constraints.

Consider a linear program:

  consider decision variables `x = [x1,x2,...,xn]`.
  
  maximize the objective function `c @ x` where `c` is given by `c = [c1,c2,...,cn]`

  subject to
  
  inequality constraints: `A @ x <= b` for an `r x n` matrix `A`
    (so `r` is the number of inequality constraints)

  equality constraints: `E @ x = f` for an `s x n` matrix `E`
    (so `s` is the number of equality constraints)
    
  with bounds: `x >= 0`

The dual of this linear program is then given by:

  consider decision variables `y = [y1,y2,...,yr]` and `z = [z1,z2,...,zs]`

  minimize the objective function `b @ y + f @ z`

  subject to `A.T @ y + E.T @ z >= c`

  with bounds: `y >= 0` and `z` free.


**Remark** In linear algebra notation, we really should write `b.T @ y + f.T @ z` for the dual objective function. 
But the way `numpy` works doesn't distinguish between row and column vectors, and there is really only one operation 
we could mean, so we give the statement above. Be aware that you may say it written differently in references!!

Let's give an example illustrating why this notion of duality makes some sense.

## LP Duality Example: Resource Allocation

### Primal Problem

A factory makes two products, $x_1$ (tables) and $x_2$ (chairs), with:

- 6 units of wood available (inequality: can leave some unused)
- Exactly 4 worker-hours must be used (equality: workers are paid regardless and so must be given work)

$$\max \quad 5x_1 + x_2$$

subject to

$$2x_1 + x_2 \leq 6 \qquad \text{(wood)}$$
$$x_1 + x_2 = 4 \qquad \text{(production quota)}$$
$$x_1, x_2 \geq 0$$

### Dual Problem

Introduce $y_1 \geq 0$ for the wood constraint (inequality) and $y_2 \in \mathbb{R}$ for the production quota constraint (equality):

$$\min \quad 6y_1 + 4y_2$$

subject to

$$2y_1 + y_2 \geq 5$$
$$y_1 + y_2 \geq 1$$
$$y_1 \geq 0, \quad y_2 \text{ free}$$



In [37]:
import numpy as np
from scipy.optimize import linprog

# formulate and solve the primal LP

c = np.array([5,1])
A = np.array([[2,1]])   # 1 x 2 matrix
b = np.array([6])
E = np.array([[1,1]])   # 1 x 2 matrix
f = np.array([4])

bounds = [(0,None)]*2

result = linprog((-1)*c,
            A_ub=A,
            b_ub=b,
            A_eq=E,
            b_eq=f,
            bounds=bounds)

def report(result):
    print(f"primal optimum: {result.fun}")
    print(f"x1 = {result.x[0]}")
    print(f"x2 = {result.x[1]}")

report(result)

primal optimum: -12.0
x1 = 2.0
x2 = 2.0


In [38]:
# formulate and solve the dual LP

r,_ = A.shape #  r=1
s,_ = E.shape #  s=1

obj_dual = np.concatenate([b, f])
A_dual = np.hstack([A.T, E.T])   # A.T is 2x1 and E.T is 2x1  
                                 #-- hstack is a "horizontally stack" procedure
                                 # so A_dual is 2x2

bounds_dual = [(0, None)] * r + [(None, None)] * s

result_dual = linprog(obj_dual, 
                      A_ub=-A_dual, 
                      b_ub=-c, 
                      bounds=bounds_dual)

# negation because linprog expects A_ub @ w <= b_ub, but we need >= c


def report_dual(result):
    print(f"dual optimum: {result.fun}")
    print(f"y = {result.x[0]}")
    print(f"z = {result.x[1]}")

report_dual(result_dual)

dual optimum: 12.0
y = 4.0
z = -3.0



Optimum value is 12.0


### Interpretation

$y_1 = 4 > 0$: wood is valuable. An extra unit of wood would let us make more tables,
increasing profit by 4.

$y_2 = -3 < 0$: the production quota is a **burden**. Being forced to produce one more unit
means making one more chair (since tables are already constrained by wood), but chairs are
nearly worthless while making them consumes wood that could have gone to tables.
The equality constraint hurts us, so its shadow price is negative.

This is why $y_2$ must be free: depending on the problem data, a fixed-resource equality
constraint can help or hurt, and the sign of its shadow price reflects which.

### Verification by perturbation

Increasing wood from 6 to 7: optimum rises by $y_1 = 4$. 

Increasing quota from 4 to 5: optimum *falls* by $y_2 = -3$, i.e. changes by $-3$. 
Decreasing quota from 4 to 3: optimum *rises* by $3$. 

The wood shadow price could not have been negative by this logic — relaxing an inequality
constraint can never hurt. But the labor shadow price has no such restriction.

In [39]:

result_wood = linprog((-1)*c,
                A_ub=A,
                b_ub=b + np.array([1]),  # increase wood by  - from 6 to 7
                A_eq=E,
                b_eq=f,
                bounds=bounds)

report(result_wood)

primal optimum: -16.0
x1 = 3.0
x2 = 1.0


In [45]:
obj_dual_quota_up = np.concatenate([b, f + np.array([1])])

result_dual_quota_up = linprog(obj_dual_quota_up, A_ub=-A_dual, b_ub=-c, bounds=bounds_dual)
report_dual(result_dual_quota_up)

dual optimum: 9.0
y = 4.0
z = -3.0


In [46]:
obj_dual_quota_dn = np.concatenate([b, f - np.array([1])])

result_dual_quota_dn = linprog(obj_dual_quota_dn,
                          A_ub=A,
                          b_ub=b,
                          A_eq=E,
                          b_eq= f,
                          bounds=bounds)

report(result_dual_quota_dn)

primal optimum: 12.0
x1 = 0.0
x2 = 4.0
